In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import torch
import re

In [2]:
# Configuration
model_name = "sdadas/polish-gpt2-medium"
random_seed = 42

# Set seed for reproducibility
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [4]:
def get_clean_half(text):
    half_length = len(text) // 2

    # Szukanie najbliższego separatora zdania przed połową tekstu
    match = re.search(r'[.!?]\s', text[:half_length][::-1])  # Przeszukiwanie od końca połowy
    if match:
        end_idx = half_length - match.start()  # Indeks końca zdania
        return text[:end_idx].strip()
    
    # Jeśli brak separatora zdania, szukaj końca pełnego słowa
    words = text[:half_length].rsplit(' ', 1)
    return words[0].strip() if len(words) > 1 else text[:half_length].strip()


# Funkcja przycinająca do końca ostatniego zdania
def truncate_to_sentence_end(text):
    # Dopasowujemy wszystkie pełne zdania
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    # Zwracamy tekst do ostatniego pełnego zdania
    return ' '.join(sentences[:-1]) if len(sentences) > 1 else text


def generate_fixed_length_completion(text, augmentation_factor=1, temperature=0.7, top_p=0.9, top_k=50, buffor=10):
    completions = []
    
    text_tokens = tokenizer.encode(text, return_tensors="pt")
    original_token_count = len(text_tokens[0])  # Liczba tokenów w oryginalnym tekście

    prompt = get_clean_half(text)
    prompt_tokens = tokenizer.encode(prompt, return_tensors="pt")
    prompt_token_count = len(prompt_tokens[0])  # Liczba tokenów w promptu

    # Obliczenie maksymalnej liczby tokenów do wygenerowania
    max_tokens_to_generate = original_token_count - prompt_token_count

    attention_mask = torch.ones_like(prompt_tokens)
    
    for _ in range(augmentation_factor):
        generated_ids = model.generate(
            prompt_tokens,
            attention_mask = attention_mask,
            max_length=prompt_token_count + max_tokens_to_generate + buffor,  # Całkowity limit tokenów
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            num_return_sequences=1
        )

        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        continuation = generated_text[len(prompt):].strip()
        continuation = truncate_to_sentence_end(continuation)
        completions.append(continuation)
    
    return completions


# Funkcja augmentacji całego zbioru danych przez dokańczanie o dokładnej długości
def augment_dataset_with_gpt2(df, text_column="text", label_column="label", augmentation_factor=1, temperature=0.7, top_p=0.9, top_k=50):
    augmented_rows = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Augmenting dataset", unit="row"):
        original_text = row[text_column]
        label = row[label_column]

        completions = generate_fixed_length_completion(
            original_text,
            augmentation_factor=augmentation_factor,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        augmented_rows.append({text_column: original_text, label_column: label})
        for completion in completions:
            prompt = get_clean_half(original_text)
            augmented_text = prompt + " " + completion
            augmented_rows.append({text_column: augmented_text, label_column: label})

    return pd.DataFrame(augmented_rows)

In [5]:
def remove_spaces_before_punctuation(text):
    # Usuwanie spacji przed kropkami i przecinkami
    return re.sub(r'\s+(?=[.,])', '', text)

In [6]:
data_path = "all_texts.csv"
df = pd.read_csv(data_path)
df['text'] = df['text'].apply(remove_spaces_before_punctuation)

augmented_df = augment_dataset_with_gpt2(
    df, 
    augmentation_factor=2,
    temperature=0.9,
    top_p=0.9,
    top_k=50
)

augmented_df.to_csv("augmented_texts_gpt2.csv", index=False)
print("Augmented dataset saved to 'augmented_texts.csv'.")

Augmenting dataset: 100%|██████████| 300/300 [09:09<00:00,  1.83s/row]

Augmented dataset saved to 'augmented_texts.csv'.


In [7]:
augmented_df.head(100)

,text,label
0,"Lakier roweru bardzo kiepskiej jakości, robią ...",Negative
1,"Lakier roweru bardzo kiepskiej jakości, robią ...",Negative
2,"Lakier roweru bardzo kiepskiej jakości, robią ...",Negative
3,Nie jestem zadowolony z zakupu. Przede wszystk...,Negative
4,Nie jestem zadowolony z zakupu. Przede wszystk...,Negative
...,...,...
95,"Ten router to porażka, gdyż nie jest w stanie ...",Negative
96,"Chyba spodziewała m się czegoś więcej, po kosm...",Neutral
97,"Chyba spodziewała m się czegoś więcej, po kosm...",Neutral
98,"Chyba spodziewała m się czegoś więcej, po kosm...",Neutral
